# テストの実行

In [119]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

In [120]:
pd.set_option('display.max_columns', None)

In [121]:
TESTSET_DIR = os.path.join('/home', 'jovyan', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jovyan', 'work')

## マスタの読み込み

In [122]:
#pre_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '01_testset_pre_master.csv'), usecols=['seed'])
sys_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '01_testset_sys_master.csv'), usecols=['seed'])
#stress_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '03_testset_stress_master.csv'), usecols=['seed'])
#param_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '04_testset_param_master.csv'), usecols=['seed'])

## 相対スコア用にChampionDataの読み込み

In [123]:
CHAMP_TAG = '20230608_2253'
CHAMP_DIR = os.path.join('/home', 'jovyan', 'work', 'result', 'champion')

champ_path = os.path.join(CHAMP_DIR, 'champ_{}.csv'.format(CHAMP_TAG))
champ_df = pd.read_csv(champ_path)

top_rate = 1.00  # 順位表を参考にチャンピオンスコアを補正
champ_score_dict = {}

for _, row in champ_df.iterrows():
    seed = row['seed']
    score = row['champion_score']

    champ_score_dict[seed] = score

## 実行するロジックの指定

In [124]:
# 実行プログラムにタグをつけておく
PROG_TAG = 'Greedy'

In [125]:
def solve(seed, prog_path):
    problem_path = os.path.join(TESTSET_DIR, 'in', '{:0>4}.txt'.format(seed)) 
    command_str = 'echo {} | {}'.format(problem_path, prog_path)

    start_time = time.perf_counter()
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    
    #print('{}'.format(prob_id))    
    return (seed, e_time, res)

In [126]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')

    testset_path = os.path.join(TESTSET_DIR, testset_name + '_master.csv')
    testset_df = pd.read_csv(testset_path)

    with futures.ThreadPoolExecutor(max_workers=24) as executor:
        for _, row in testset_df.iterrows():
            seed, problem = row

            # バッチ実行
            future = executor.submit(solve, seed=seed, prog_path=prog_path)
            future_list.append(future)

        _ = futures.as_completed(fs=future_list)

    for future in future_list:
        seed, e_time, res = future.result()

        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)

        # 経過時間
        solve_result.append(e_time)
        
        try:
            elem_cnt = 2
            
            # Result
            result = res.stderr.decode('utf-8').split()[-elem_cnt].replace('Result=', '')

            # Score
            score = int(res.stderr.decode('utf-8').split()[-elem_cnt+1].replace('Score=', ''))
            
            # 相対スコア
            # rel_score = int(10 ** 9 * top_rate * champ_score_dict[seed] / score)

        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        solve_result.append(score)
        # solve_result.append(rel_score)

        solve_result.append(result)

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    result_df.columns = ['testset', 'seed', 'time', 'score', 'result']

    # 自己相対スコアを算出
    
    return result_df

In [127]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [128]:
def get_summary_df(result_df):
    # 全体サマリ
    summary_all_df = pd.DataFrame()

    for testset in np.unique(result_df['testset']):
        test_result_df = result_df.query('testset == "{}"'.format(testset))

        summary_df = pd.DataFrame(
        {
            'testset': [testset],
            
            'time_mean': [int(np.mean(test_result_df['time']))],
            
            'score_mean': [int(np.mean(test_result_df['score']))],
            'score_min': [min(test_result_df['score'])],
            'score_max': [max(test_result_df['score'])],

            'time_max': [max(test_result_df['time'])],
        })

        summary_all_df = pd.concat([summary_all_df, summary_df], axis=0)   

    summary_all_df['tag'] = PROG_TAG
    
    cols = ['tag']
    cols.extend(summary_df.columns)
    
    summary_all_df = summary_all_df[cols]
    
    return summary_all_df

In [129]:
PROG_NAME_LIST = ['main']
#PROG_NAME_LIST = ['main', 'main_off']

testset_name = '01_testset_sys'
#testset_name = '02_testset_sys'
#testset_name = '03_testset_stress'
#testset_name = '04_testset_param'

result_dict = {}
summary_all_dict = {}

for PROG_NAME in PROG_NAME_LIST:
    prog_path = os.path.join(PRJ_DIR, PROG_NAME)
    
    result_df = pd.DataFrame()
    
    testset_result_df = run_test(testset_name)
    result_df = pd.concat([result_df, testset_result_df], axis=0)
    
    result_dict[PROG_NAME] = result_df
    summary_all_dict[PROG_NAME] = get_summary_df(result_df)    

2023-06-08 14:31:02,190: Start
2023-06-08 14:31:02,430: finish!


In [131]:
# pre test
result_sub_df = pd.merge(sys_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '01_test_pre'

summary_df

,tag,testset,time_mean,score_mean,score_min,score_max,time_max
0,Greedy,01_test_pre,16,349287,178550,585538,32


In [132]:
result_df.sort_values('score', ascending=True)

,testset,seed,time,score,result
141,01_testset_sys,141,14,178550,LLLLFLLLFLLFLLBLFLFLLBLBLFFLFLLLBLLLLLBFLBLFLF...
49,01_testset_sys,49,13,193872,LLLLLFLFLLLBRBRRFFFRFFFFRFRRRLFRFFLLLFLLFLFRFF...
104,01_testset_sys,104,14,198500,LLFLFFLLLLLLLFRLLLLLFLLFLLLLLLLLLLLLLLLFLRFFRR...
75,01_testset_sys,75,16,202274,LLLLFLLRRFLLLLFBRRRBRRBBRBBRRRRRRRRRRRRRBRRRBR...
172,01_testset_sys,172,15,222288,LLFLFFLFLLFRLLFLFLBBLLLLLLRFFFFFFFRFRBRBBBRRRF...
...,...,...,...,...,...
26,01_testset_sys,26,20,529022,LLFLLBLLLBRRRFBRRFLFLFLBLLLLBLLLBLBBBLLLLLLLLL...
13,01_testset_sys,13,10,539015,LFLLLFLFLRRRFFRFLFLLLLLLLBBBLLLLBLLLLLBLLLLLLL...
56,01_testset_sys,56,15,555619,LLFLFLBBLLLFLLFFRLLRFFFFFFFFRFFFLLLLBLLLLLLLLL...
63,01_testset_sys,63,18,576252,LLLLLLLLLLLFLLBLLLLLLLFRRRLLFFRRFFFFLLLLLLFLLB...


In [133]:
result_df.sort_values('time', ascending=True)

,testset,seed,time,score,result
0,01_testset_sys,0,8,332540,LLFLBLFLLLLLLLLFLBLLLLBLLLLLBLLLLBLLRRRBRLBBBL...
198,01_testset_sys,198,8,312646,LLLFLFLLLFLLFFFLLFLLFLLLLLLLFRBLBLBLLBBBBBLBLL...
197,01_testset_sys,197,9,497933,LLLLLLBLFLRRRFLLLLLLLFRRRRRRRRRRRFFLFLLLLLLLLF...
196,01_testset_sys,196,9,383244,LLLLFLLLFLLFRFRFFRRRRRFFLLLLLFLLFFLLLFFFLLLLLL...
191,01_testset_sys,191,9,359049,LLFLLLLLFLLLLFLLLLLBRLLLLBLBLLBFLLBFLBLBLLFFLF...
...,...,...,...,...,...
50,01_testset_sys,50,26,314560,LFLLLFLLLLBLLLBRRRBRFRRRRRRRRFFBRRBRBRRRRRBLFR...
42,01_testset_sys,42,27,483852,LLLLLFLLLLLLLLLLFRBRFBBRRRRRRBRRFFRFFFFFLLLLLF...
30,01_testset_sys,30,30,260386,LLLLRBRLFLLBLBLLLLLLLLLLBLRRBRBRRRRRRRRRRRRRRR...
28,01_testset_sys,28,30,375804,LLLLLLLFLLFRFFFLLFLLLBLLLBLLLLLFLFLLBBBLBLBBBL...


In [134]:
result_df.sort_values('rel_score', ascending=True).head(n=10)

KeyError: 'rel_score'

# 結果ログの保存

In [135]:
t_now = datetime.datetime.now() + datetime.timedelta(hours=9)
time_str = t_now.strftime('%Y%m%d_%H%M')

for PROG_NAME in PROG_NAME_LIST:
    result_df = result_dict[PROG_NAME]
    
    for testset in np.unique(result_df['testset']):
        csv_df = result_df.query('testset == "{}"'.format(testset))
        csv_df.to_csv(PRJ_DIR+'/result/{}_{}_{}_{}.csv'.format(time_str,PROG_TAG, testset, PROG_NAME), index=False)